## Initialise Spark Session:

In [1]:
val spark = org.apache.spark.sql.SparkSession.builder
        .master("local") 
        .appName("Spark CSV Reader")
        .getOrCreate;

Intitializing Scala interpreter ...

Spark Web UI available at http://66a241763ee5:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1590132004514)
SparkSession available as 'spark'


2020-05-22 07:20:11,139 WARN  [Thread-4] sql.SparkSession$Builder (Logging.scala:logWarning(66)) - Using an existing SparkSession; some configuration may not take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@59d963ab


### Preparing HDFS

In [2]:
!pwd

/home/sandpit/big-data-realestate/scripts



In [3]:
!cat ./../data-raw/Melbourne_housing_FULL.csv| wc -l

34858



In [4]:
! hadoop fs -mkdir -p  /tmp/rs_in
! hadoop fs -put   -p  ./../data-raw/Melbourne_housing_FULL.csv             /tmp/rs_in/mh.csv
! hadoop fs -ls        /tmp/rs_in/

Found 1 items


-rw-r--r--   1 root root    5018236 2020-05-15 05:20 /tmp/rs_in/mh.csv




In [5]:
!hadoop fs -cat /tmp/rs_in/mh.csv | wc -l

34858



### Get config info about hdfs:

In [6]:
!hdfs getconf -confKey fs.defaultFS

hdfs://localhost:9000



In [7]:
val df = spark.read.format("csv").option("header", "true").load("hdfs://localhost:9000/tmp/rs_in/mh.csv")

df: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


### Print schema:

In [8]:
df.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Car: string (nullable = true)
 |-- Landsize: string (nullable = true)
 |-- BuildingArea: string (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)



In [9]:
df.columns

res1: Array[String] = Array(Suburb, Address, Rooms, Type, Price, Method, SellerG, Date, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea, YearBuilt, CouncilArea, Lattitude, Longtitude, Regionname, Propertycount)


### Show column types:

In [10]:
df.dtypes

res2: Array[(String, String)] = Array((Suburb,StringType), (Address,StringType), (Rooms,StringType), (Type,StringType), (Price,StringType), (Method,StringType), (SellerG,StringType), (Date,StringType), (Distance,StringType), (Postcode,StringType), (Bedroom2,StringType), (Bathroom,StringType), (Car,StringType), (Landsize,StringType), (BuildingArea,StringType), (YearBuilt,StringType), (CouncilArea,StringType), (Lattitude,StringType), (Longtitude,StringType), (Regionname,StringType), (Propertycount,StringType))


### Display first 12 columns:

In [11]:
df.select("Suburb","Address","Rooms","Type","Price","Method","SellerG","Date","Distance","Postcode","Bathroom","Car").show()

+----------+-------------------+-----+----+-------+------+-------+---------+--------+--------+--------+----+
|    Suburb|            Address|Rooms|Type|  Price|Method|SellerG|     Date|Distance|Postcode|Bathroom| Car|
+----------+-------------------+-----+----+-------+------+-------+---------+--------+--------+--------+----+
|Abbotsford|      68 Studley St|    2|   h|   null|    SS| Jellis|3/09/2016|     2.5|    3067|       1|   1|
|Abbotsford|       85 Turner St|    2|   h|1480000|     S| Biggin|3/12/2016|     2.5|    3067|       1|   1|
|Abbotsford|    25 Bloomburg St|    2|   h|1035000|     S| Biggin|4/02/2016|     2.5|    3067|       1|   0|
|Abbotsford| 18/659 Victoria St|    3|   u|   null|    VB| Rounds|4/02/2016|     2.5|    3067|       2|   1|
|Abbotsford|       5 Charles St|    3|   h|1465000|    SP| Biggin|4/03/2017|     2.5|    3067|       2|   0|
|Abbotsford|   40 Federation La|    3|   h| 850000|    PI| Biggin|4/03/2017|     2.5|    3067|       2|   1|
|Abbotsford|       

### Display last 8 columns:

In [12]:
df.select("Landsize","BuildingArea","YearBuilt","CouncilArea","Lattitude","Longtitude","Regionname","Propertycount").show(10)

+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|Landsize|BuildingArea|YearBuilt|       CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|     126|        null|     null|Yarra City Council| -37.8014|  144.9958|Northern Metropol...|         4019|
|     202|        null|     null|Yarra City Council| -37.7996|  144.9984|Northern Metropol...|         4019|
|     156|          79|     1900|Yarra City Council| -37.8079|  144.9934|Northern Metropol...|         4019|
|       0|        null|     null|Yarra City Council| -37.8114|  145.0116|Northern Metropol...|         4019|
|     134|         150|     1900|Yarra City Council| -37.8093|  144.9944|Northern Metropol...|         4019|
|      94|        null|     null|Yarra City Council| -37.7969|  144.9969|Northern Metropol...|         4019|
|     120|         

In [13]:
df.describe().select("summary","Price", "Rooms","Distance","Bathroom","Car").show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|            Price|             Rooms|          Distance|          Bathroom|               Car|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|            27247|             34857|             34857|             26631|             26129|
|   mean|1050173.344955408|3.0310124221820582|11.184929423916007| 1.624798167549097|1.7288453442535114|
| stddev|641467.1301045999|0.9699329348975204| 6.788892455935938|0.7242120114699068|1.0107707853554244|
|    min|          1000000|                 1|              #N/A|                 0|                 0|
|    max|           999999|                 9|               9.9|                 9|                 9|
+-------+-----------------+------------------+------------------+------------------+------------------+



In [14]:
df.describe().select("summary","Price","Landsize", "BuildingArea").show()

+-------+-----------------+------------------+------------------+
|summary|            Price|          Landsize|      BuildingArea|
+-------+-----------------+------------------+------------------+
|  count|            27247|             23047|             13742|
|   mean|1050173.344955408|  593.598993361392| 160.2564003565711|
| stddev|641467.1301045999|3398.8419464599056|401.26706008485496|
|    min|          1000000|                 0|                 0|
|    max|           999999|               999|               999|
+-------+-----------------+------------------+------------------+



#### Change "#N/A" to null

In [15]:
df.columns

res7: Array[String] = Array(Suburb, Address, Rooms, Type, Price, Method, SellerG, Date, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea, YearBuilt, CouncilArea, Lattitude, Longtitude, Regionname, Propertycount)


In [16]:
var df_result = df
for (colName <- df_result.columns){ 
  df_result = df_result.withColumn(colName, when(trim(df_result(colName)).equalTo("#N/A"),null).otherwise(df_result(colName)))
  }

df_result: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


In [17]:
df_result.describe().select("summary","Price", "Rooms","Distance","Bathroom","Car").show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|            Price|             Rooms|          Distance|          Bathroom|               Car|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|            27247|             34857|             34856|             26631|             26129|
|   mean|1050173.344955408|3.0310124221820582|11.184929423916007| 1.624798167549097|1.7288453442535114|
| stddev|641467.1301045999|0.9699329348975204| 6.788892455935938|0.7242120114699068|1.0107707853554244|
|    min|          1000000|                 1|                 0|                 0|                 0|
|    max|           999999|                 9|               9.9|                 9|                 9|
+-------+-----------------+------------------+------------------+------------------+------------------+



#### Convert numeric data represented as string into double 

In [18]:
val doubleColNames = df_result.select("Price", "Rooms","Bedroom2","Distance","Bathroom","Car", "Landsize", "BuildingArea","Propertycount",
               "YearBuilt","Lattitude", "Longtitude").columns
//val colNames =df.columns
for (colName<-doubleColNames){ 
    df_result=df_result.withColumn(colName,col(colName).cast("Double"))
}

doubleColNames: Array[String] = Array(Price, Rooms, Bedroom2, Distance, Bathroom, Car, Landsize, BuildingArea, Propertycount, YearBuilt, Lattitude, Longtitude)


In [19]:
df_result.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: double (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: double (nullable = true)



### Filtering

In [20]:
df_result.filter($"Suburb"==="Glen Waverley").select("Address","Rooms","Type","Price","Method","SellerG","Date","Distance","Postcode","Bathroom","Car").show()

+-----------------+-----+----+---------+------+---------+----------+--------+--------+--------+----+
|          Address|Rooms|Type|    Price|Method|  SellerG|      Date|Distance|Postcode|Bathroom| Car|
+-----------------+-----+----+---------+------+---------+----------+--------+--------+--------+----+
|     7 Marbray Dr|  4.0|   h|     null|    SN|Harcourts| 1/07/2017|    16.7|    3150|     1.0| 2.0|
|      24 Owens Av|  4.0|   h|1250000.0|     S|      Ray| 1/07/2017|    16.7|    3150|    null|null|
|515 Springvale Rd|  3.0|   h|     null|    PI|      Ray| 1/07/2017|    16.7|    3150|     1.0| 2.0|
| 22 Stableford Av|  3.0|   h|     null|     S|      Ray| 1/07/2017|    16.7|    3150|    null|null|
|  28 Brentwood Dr|  5.0|   h|     null|    PI|      Ray| 3/06/2017|    16.7|    3150|     5.0| 2.0|
|2/70 Leicester Av|  3.0|   t|     null|    SP|      LLC| 3/06/2017|    16.7|    3150|    null|null|
|    38 Margate Cr|  3.0|   h|     null|    SN| Woodards| 3/06/2017|    16.7|    3150|    n

In [21]:
df_result.where("Price >1000000").filter("Suburb = 'Abbotsford'").select("Address","Rooms","Type","Price","Method","SellerG","Date","Distance","Postcode","Bathroom","Car", "Propertycount").count()

res13: Long = 53


In [22]:
df_result.where("Price >1000000").filter("Suburb = 'Abbotsford'").select("Address","Rooms","Type","Price","Method","SellerG","Date","Distance","Postcode","Bathroom","Car", "Propertycount").show()

+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+----+-------------+
|            Address|Rooms|Type|    Price|Method| SellerG|      Date|Distance|Postcode|Bathroom| Car|Propertycount|
+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+----+-------------+
|       85 Turner St|  2.0|   h|1480000.0|     S|  Biggin| 3/12/2016|     2.5|    3067|     1.0| 1.0|       4019.0|
|    25 Bloomburg St|  2.0|   h|1035000.0|     S|  Biggin| 4/02/2016|     2.5|    3067|     1.0| 0.0|       4019.0|
|       5 Charles St|  3.0|   h|1465000.0|    SP|  Biggin| 4/03/2017|     2.5|    3067|     2.0| 0.0|       4019.0|
|        55a Park St|  4.0|   h|1600000.0|    VB|  Nelson| 4/06/2016|     2.5|    3067|     1.0| 2.0|       4019.0|
|       124 Yarra St|  3.0|   h|1876000.0|     S|  Nelson| 7/05/2016|     2.5|    3067|     2.0| 0.0|       4019.0|
|      98 Charles St|  2.0|   h|1636000.0|     S|  Nelson| 8/10/2016|   

In [23]:
df.dtypes.filter(colTup => colTup._1 == "Suburb")

res15: Array[(String, String)] = Array((Suburb,StringType))


In [24]:
df_result.select("Address","Type","Method","SellerG","Postcode","CouncilArea","Regionname").count()

res16: Long = 34857


### Categorical Attributes

#### 1. Address

In [25]:
// rename into street keep only street name
import org.apache.spark.sql.functions.countDistinct
df_result.select("Address").distinct.show()

+-------------------+
|            Address|
+-------------------+
|      557 Orrong Rd|
|      19 Poulter St|
|    43 Riverside Av|
|       11 South Tce|
|  41 Marlborough St|
|          4 Park Cr|
|        3/3 Dega Av|
|        93 Tudor St|
|         10 Kent Rd|
|       18 Thomas St|
|   1/1 Glen Iris Rd|
|      7 Allambee Av|
|    83 Truganini Rd|
|       130 Keele St|
|       8 Winters Wy|
|     36a Mitford St|
|   7/223 Station St|
|1/146 Ascot Vale Rd|
|    5/60 Farnham St|
|      22 Renwick St|
+-------------------+
only showing top 20 rows



import org.apache.spark.sql.functions.countDistinct


In [26]:
df_result.filter("Address IS NULL").count()

res18: Long = 0


In [27]:
df_result.select("Address").distinct.count()

res19: Long = 34009


In [28]:
df_result.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: double (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: double (nullable = true)



#### Split Address on Street and Suffix

In [29]:
//split address on Street and Suffix
df_result = df_result.withColumn("Street",split(col("Address")," ").getItem(1)).
                   withColumn("Suffix",split(col("Address")," ").getItem(2)).drop("Address")
df_result.show()

+----------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+----+--------+------------+---------+------------------+---------+----------+--------------------+-------------+----------+------+
|    Suburb|Rooms|Type|    Price|Method|SellerG|     Date|Distance|Postcode|Bedroom2|Bathroom| Car|Landsize|BuildingArea|YearBuilt|       CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|    Street|Suffix|
+----------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+----+--------+------------+---------+------------------+---------+----------+--------------------+-------------+----------+------+
|Abbotsford|  2.0|   h|     null|    SS| Jellis|3/09/2016|     2.5|    3067|     2.0|     1.0| 1.0|   126.0|        null|     null|Yarra City Council| -37.8014|  144.9958|Northern Metropol...|       4019.0|   Studley|    St|
|Abbotsford|  2.0|   h|1480000.0|     S| Biggin|3/12/2016|     2.5|    3067|     2.0|     1.0| 1.0| 

df_result: org.apache.spark.sql.DataFrame = [Suburb: string, Rooms: double ... 20 more fields]


In [30]:
df_result.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Rooms: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: double (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Suffix: string (nullable = true)



In [31]:
df_result.filter("Street IS NULL")

res23: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Suburb: string, Rooms: double ... 20 more fields]


In [32]:
val streets = df_result.select("Street").distinct()
streets.count()

streets: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Street: string]
res24: Long = 7494


In [33]:
streets.show()

+----------+
|    Street|
+----------+
|  Laurence|
|  Farncomb|
|     Abbin|
|  Moreland|
|   Hanover|
|  Doonkuna|
|Pilkington|
| Bangalore|
|  Disraeli|
|     Ewart|
|   Clynden|
|     Lorne|
|      Faye|
|    Maddox|
|   Pickles|
|     Tyler|
|  Crevelli|
|      Silk|
|   Palermo|
|   Boreham|
+----------+
only showing top 20 rows



#### 2. Postcode

In [34]:
var postcodes = df_result.select("Postcode").distinct()

postcodes: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Postcode: string]


In [35]:
df_result.filter("Postcode IS NULL").count()

res26: Long = 1


In [36]:
postcodes.count()

res27: Long = 212


#### 3. Suburb

In [37]:
val suburbs = df_result.select("Suburb").distinct

suburbs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Suburb: string]


In [38]:
suburbs.count()

res28: Long = 351


In [39]:

df.filter("Suburb IS NULL").count()

res29: Long = 0


In [40]:
// make first letter of suburb upper case
import org.apache.spark.sql.functions._
df_result = df_result.withColumn("Suburb", initcap(col("Suburb")))
val suburbs = df_result.select("Suburb").distinct
suburbs.count()

import org.apache.spark.sql.functions._
df_result: org.apache.spark.sql.DataFrame = [Suburb: string, Rooms: double ... 20 more fields]
suburbs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Suburb: string]
res30: Long = 349


In [41]:
suburbs.show()

+----------------+
|          Suburb|
+----------------+
|  Brunswick West|
| South Melbourne|
|    Ivanhoe East|
|    Princes Hill|
|      Cranbourne|
|         Ashwood|
|       Brunswick|
|South Kingsville|
|        Brighton|
|        Oak Park|
|         Doveton|
|       Albanvale|
|      Brookfield|
|        Lynbrook|
|     Ferny Creek|
|     Pascoe Vale|
| Blackburn North|
|     Sandringham|
|   Botanic Ridge|
|          Carrum|
+----------------+
only showing top 20 rows



#### 4. Type 
#### Distinct values 

In [42]:
import org.apache.spark.sql.functions.countDistinct
df_result.select("Type").distinct.show()

+----+
|Type|
+----+
|   h|
|   u|
|   t|
+----+



import org.apache.spark.sql.functions.countDistinct


In [43]:
df_result = df_result.withColumn("Type", initcap(col("Type")))
val types = df_result.select("Type").distinct

df_result: org.apache.spark.sql.DataFrame = [Suburb: string, Rooms: double ... 20 more fields]
types: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Type: string]


In [44]:
types.count()

res33: Long = 3


In [45]:
types.show()

+----+
|Type|
+----+
|   T|
|   U|
|   H|
+----+



#### Null values  

In [46]:
df_result.filter("Type IS NULL").count()

res35: Long = 0


#### 5. Method

In [47]:
df_result.filter("Method IS NULL").count()

res36: Long = 0


In [48]:
val methods = df_result.select("Method").distinct

methods: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Method: string]


In [49]:
methods.count()

res37: Long = 9


In [50]:
methods.show()

+------+
|Method|
+------+
|    PI|
|    SA|
|    SP|
|    VB|
|    PN|
|     W|
|     S|
|    SN|
|    SS|
+------+



#### 6. SellerG

In [51]:

df_result.filter("SellerG IS NULL").count()

res39: Long = 0


In [52]:
val sellers = df_result.select("SellerG").distinct

sellers: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [SellerG: string]


In [53]:
sellers.count()

res40: Long = 388


In [54]:
sellers.show()

+--------------------+
|             SellerG|
+--------------------+
|              LITTLE|
|                 S&L|
|              Ristic|
|            Langwell|
|             Ruralco|
|             Xynergy|
|               Ryder|
|               iSell|
|               Scott|
|              Wilson|
|          McNaughton|
|           Blackbird|
|hockingstuart/Biggin|
|               Lucas|
|                 One|
|         Buxton/Find|
|                Real|
|            Sterling|
|             Compton|
|           Tiernan's|
+--------------------+
only showing top 20 rows



#### 7. Date

In [55]:
df_result.filter("Date IS NULL").count()

res42: Long = 0


In [56]:
val dates = df_result.select("Date").distinct()

dates: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: string]


In [57]:
dates.count()

res43: Long = 78


In [58]:
dates.show()

+----------+
|      Date|
+----------+
|16/04/2016|
|29/04/2017|
|10/12/2016|
|19/08/2017|
| 7/05/2016|
| 8/07/2017|
| 4/03/2017|
|29/07/2017|
|27/05/2017|
|28/10/2017|
| 9/09/2017|
|26/07/2016|
|12/11/2016|
|25/02/2017|
| 6/05/2017|
|18/11/2017|
| 3/09/2016|
| 3/12/2016|
|25/11/2017|
| 3/06/2017|
+----------+
only showing top 20 rows



#### 8. CouncilArea

In [59]:

df_result.filter("CouncilArea IS NULL").count()

res45: Long = 3


In [60]:
val sareas = df_result.select("CouncilArea").distinct()

sareas: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CouncilArea: string]


In [61]:
sareas.count()

res46: Long = 34


In [62]:
sareas.show(34)

+--------------------+
|         CouncilArea|
+--------------------+
|Bayside City Council|
|Greater Dandenong...|
|   Hume City Council|
|Glen Eira City Co...|
|Kingston City Cou...|
|Nillumbik Shire C...|
| Monash City Council|
|Macedon Ranges Sh...|
|   Knox City Council|
|Wyndham City Council|
|                null|
|Mitchell Shire Co...|
|Maribyrnong City ...|
|Whittlesea City C...|
|Whitehorse City C...|
|Frankston City Co...|
|Manningham City C...|
|Darebin City Council|
|Moreland City Cou...|
|Cardinia Shire Co...|
|Moonee Valley Cit...|
|Boroondara City C...|
|Yarra Ranges Shir...|
|  Casey City Council|
|Port Phillip City...|
|Brimbank City Cou...|
|Hobsons Bay City ...|
|Banyule City Council|
|Stonnington City ...|
| Melton City Council|
|  Yarra City Council|
|Melbourne City Co...|
|Moorabool Shire C...|
|Maroondah City Co...|
+--------------------+



#### 9. Regionname

In [63]:
df_result.filter("Regionname IS NULL").count()

res48: Long = 3


In [64]:
val regions = df_result.select("Regionname").distinct
regions.count()

regions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Regionname: string]
res49: Long = 9


In [65]:
regions.show()

+--------------------+
|          Regionname|
+--------------------+
|South-Eastern Met...|
|Western Metropolitan|
|                null|
|Eastern Metropolitan|
|    Eastern Victoria|
|   Northern Victoria|
|Northern Metropol...|
|Southern Metropol...|
|    Western Victoria|
+--------------------+



#### 10. YearBuilt

In [66]:
df_result.filter("YearBuilt IS NULL").count()

res51: Long = 19306


In [67]:
val years = df_result.filter("YearBuilt IS NULL").distinct

years: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Suburb: string, Rooms: double ... 20 more fields]


In [68]:
years.count()

res52: Long = 19296


In [69]:
years.show()

+--------------+-----+----+---------+------+-------------+----------+--------+--------+--------+--------+----+--------+------------+---------+--------------------+---------+----------+--------------------+-------------+----------+------+
|        Suburb|Rooms|Type|    Price|Method|      SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom| Car|Landsize|BuildingArea|YearBuilt|         CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|    Street|Suffix|
+--------------+-----+----+---------+------+-------------+----------+--------+--------+--------+--------+----+--------+------------+---------+--------------------+---------+----------+--------------------+-------------+----------+------+
|  Airport West|  2.0|   T| 460000.0|    VB|       Nelson|18/06/2016|    13.5|    3042|    null|    null|null|    null|        null|     null|Moonee Valley Cit...|     null|      null|Western Metropolitan|       3464.0|  Hillside|    Gr|
|  Altona North|  3.0|   H| 700000.0|    VB|    

#### 1. Price

In [70]:
df_result.filter("Price IS NULL").count()

res54: Long = 7610


In [71]:
//get rid of null in price
df_result=df_result.filter(!df_result("Price").isNull)

df_result: org.apache.spark.sql.DataFrame = [Suburb: string, Rooms: double ... 20 more fields]


In [72]:
val prices = df_result.select("Price").distinct()

prices: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double]


In [73]:
prices.count()

res55: Long = 2871


In [74]:
prices.show()

+---------+
|    Price|
+---------+
| 300000.0|
| 495000.0|
|1185000.0|
| 330000.0|
| 532000.0|
| 940500.0|
| 452500.0|
| 640500.0|
| 431500.0|
|1155500.0|
| 474000.0|
| 546000.0|
|1483000.0|
| 671000.0|
|1055000.0|
| 677776.0|
|2053000.0|
|1259000.0|
| 975500.0|
|4690000.0|
+---------+
only showing top 20 rows



In [75]:
df_result.describe().select("summary","Price","Suburb").show()

+-------+-----------------+----------+
|summary|            Price|    Suburb|
+-------+-----------------+----------+
|  count|            27247|     27247|
|   mean|1050173.344955408|      null|
| stddev|641467.1301045999|      null|
|    min|          85000.0|Abbotsford|
|    max|           1.12E7|Yarraville|
+-------+-----------------+----------+



#### Histogram Utility Function

In [76]:
// utility function for numerical attributes:
// returns  dataframe with values,frequencies
// param DataFrame Column, number of points
def hist(df:org.apache.spark.sql.DataFrame, points:Int):org.apache.spark.sql.DataFrame ={
    val (startValues,counts) = 
      df.map(value => value.getDouble(0)).rdd.histogram(points)
   val zippedValues = startValues.zip(counts)
   val rowRDD = zippedValues.map( value => Tuple2(value._1,value._2))
   var histDf = spark.createDataFrame(rowRDD)
   histDf = histDf.withColumnRenamed("_1","value")
     .withColumnRenamed("_2","frequency")
   histDf 
}

hist: (df: org.apache.spark.sql.DataFrame, points: Int)org.apache.spark.sql.DataFrame


In [77]:
hist(df_result.select("Price"),10).show()

+---------+---------+
|    value|frequency|
+---------+---------+
|  85000.0|    19047|
|1196500.0|     6984|
|2308000.0|      944|
|3419500.0|      186|
|4531000.0|       60|
|5642500.0|       20|
|6754000.0|        3|
|7865500.0|        1|
|8977000.0|        1|
|1.00885E7|        1|
+---------+---------+



#### 1. Rooms

In [78]:
df_result.filter("Rooms IS NULL").count()

res59: Long = 0


In [79]:
val rooms = df_result.select("Rooms").distinct()

rooms: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Rooms: double]


In [80]:
rooms.count()

res60: Long = 12


In [81]:
rooms.show()

+-----+
|Rooms|
+-----+
|  8.0|
|  7.0|
|  1.0|
|  4.0|
|  3.0|
|  2.0|
| 10.0|
|  6.0|
|  5.0|
|  9.0|
| 16.0|
| 12.0|
+-----+



In [82]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



#### 2. Distance

In [83]:
df_result.filter("Distance IS NULL").count()

res63: Long = 1


In [84]:
val dists = df_result.select("Distance").distinct

dists: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Distance: double]


In [85]:
dists.count()

res64: Long = 214


In [86]:
dists.show(213)

+--------+
|Distance|
+--------+
|    14.9|
|    13.4|
|    15.5|
|    15.4|
|     2.4|
|     8.0|
|    10.2|
|    24.7|
|    32.3|
|     0.0|
|    17.9|
|    43.3|
|    11.4|
|     5.4|
|    23.8|
|    16.6|
|     7.0|
|    11.5|
|     3.5|
|    31.7|
|     6.1|
|     9.5|
|     7.7|
|    17.3|
|    25.2|
|    31.6|
|     6.6|
|    34.7|
|    20.5|
|     8.7|
|    13.3|
|    28.8|
|    12.5|
|     3.7|
|    10.3|
|     4.5|
|    19.9|
|     5.7|
|     1.4|
|    26.1|
|     6.7|
|     0.7|
|     7.4|
|     2.3|
|    13.8|
|     6.5|
|    45.2|
|    null|
|    19.6|
|     3.4|
|    21.1|
|    16.1|
|    18.0|
|    35.4|
|    20.8|
|    16.7|
|    16.5|
|    23.6|
|    12.8|
|     8.4|
|     2.5|
|    23.2|
|    39.0|
|     9.8|
|    17.5|
|    18.4|
|    12.3|
|    34.6|
|    44.2|
|    22.9|
|    12.1|
|     3.1|
|    21.3|
|    20.1|
|    13.9|
|    12.9|
|     2.7|
|    25.0|
|    14.2|
|    22.7|
|     4.1|
|    39.8|
|    12.4|
|     2.8|
|    34.9|
|    41.0|
|     9.3|
|     8.8|

In [87]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



#### 3. Bathroom

In [88]:
df_result.filter("Bathroom IS NULL").count()


res67: Long = 6447


In [89]:
val brooms = df_result.select("Bathroom").distinct

brooms: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Bathroom: double]


In [90]:
brooms.count()

res68: Long = 11


In [91]:
brooms.show()

+--------+
|Bathroom|
+--------+
|     8.0|
|     0.0|
|     7.0|
|    null|
|     1.0|
|     4.0|
|     3.0|
|     2.0|
|     6.0|
|     5.0|
|     9.0|
+--------+



In [92]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



#### 4. Car

In [93]:
df.filter("Car IS NULL").count()

res71: Long = 8728


In [94]:
val cars = df_result.select("Car").distinct

cars: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Car: double]


In [95]:
cars.count()

res72: Long = 14


In [96]:
cars.show()

+----+
| Car|
+----+
| 8.0|
| 0.0|
| 7.0|
|null|
|18.0|
| 1.0|
| 4.0|
|11.0|
| 3.0|
| 2.0|
|10.0|
| 6.0|
| 5.0|
| 9.0|
+----+



In [97]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



#### 5. Landsize

In [98]:
df_result.filter("Landsize IS NULL").count()

res75: Long = 9265


In [99]:
val lands = df_result.select("Landsize").distinct

lands: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Landsize: double]


In [100]:
lands.count()

res76: Long = 1558


In [101]:
lands.show()

+--------+
|Landsize|
+--------+
|   596.0|
|   558.0|
|   299.0|
|   692.0|
|   305.0|
|   496.0|
|   934.0|
|   769.0|
|  3901.0|
|  1051.0|
|   147.0|
|   170.0|
|   576.0|
|   782.0|
|   720.0|
|   184.0|
|   810.0|
|  1369.0|
|  3826.0|
|  1587.0|
+--------+
only showing top 20 rows



In [102]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



#### 6. BuildingArea

In [103]:
df_result.filter("BuildingArea IS NULL").count()

res79: Long = 16591


In [104]:
val bareas = df_result.select("BuildingArea").distinct

bareas: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [BuildingArea: double]


In [105]:
bareas.count()

res80: Long = 663


In [106]:
bareas.show()

+------------+
|BuildingArea|
+------------+
|       305.0|
|       558.0|
|       934.0|
|       496.0|
|       147.0|
|       170.0|
|       720.0|
|       184.0|
|    140.7481|
|       169.0|
|       160.0|
|        72.3|
|        53.3|
|        70.0|
|       311.0|
|        67.0|
|       168.0|
|        69.0|
|       206.0|
|         0.0|
+------------+
only showing top 20 rows



In [107]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



#### 7. Propertycount

In [108]:
df_result.filter("Propertycount IS NULL").count()

res83: Long = 3


In [109]:
val pcounts = df_result.select("Propertycount").distinct

pcounts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Propertycount: double]


In [110]:
pcounts.count()

res84: Long = 337


In [111]:
pcounts.show(639)

+-------------+
|Propertycount|
+-------------+
|       1369.0|
|       4385.0|
|        810.0|
|        608.0|
|       3640.0|
|      11925.0|
|       4718.0|
|        973.0|
|       1554.0|
|        389.0|
|       2954.0|
|        249.0|
|        984.0|
|       1130.0|
|        902.0|
|       3755.0|
|        802.0|
|       2417.0|
|       5051.0|
|        438.0|
|       6065.0|
|      11308.0|
|       3284.0|
|        588.0|
|       8524.0|
|       4497.0|
|      10160.0|
|        271.0|
|       3600.0|
|       4654.0|
|       6938.0|
|       4973.0|
|       2949.0|
|       1123.0|
|       2291.0|
|       1345.0|
|       1475.0|
|       6380.0|
|       3224.0|
|       8743.0|
|       6464.0|
|       1607.0|
|       6795.0|
|       9028.0|
|      11364.0|
|      17384.0|
|       7082.0|
|       4707.0|
|       4168.0|
|       6388.0|
|       7822.0|
|       6990.0|
|      10788.0|
|       1624.0|
|       5556.0|
|        538.0|
|       4704.0|
|       2555.0|
|       8989.0|
|       

In [112]:
hist(df_result.select("Rooms"),10).show()

+-----+---------+
|value|frequency|
+-----+---------+
|  1.0|     7983|
|  2.5|    11957|
|  4.0|     7127|
|  5.5|      137|
|  7.0|       34|
|  8.5|        1|
| 10.0|        5|
| 11.5|        2|
| 13.0|        0|
| 14.5|        1|
+-----+---------+



In [113]:
df_result.describe().select("summary","Price", "Rooms","Distance","Bathroom","Car").show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|            Price|             Rooms|          Distance|          Bathroom|               Car|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|            27247|             27247|             27246|             20800|             20423|
|   mean|1050173.344955408|2.9922927294748045|11.280276003817034|1.5916826923076923|1.7153699260637516|
| stddev|641467.1301045999|0.9547952950194317| 6.787468996334042|0.7008803646354179| 0.994216095184362|
|    min|          85000.0|               1.0|               0.0|               0.0|               0.0|
|    max|           1.12E7|              16.0|              48.1|               9.0|              18.0|
+-------+-----------------+------------------+------------------+------------------+------------------+



In [114]:
df_result.describe().select("summary","Landsize","Propertycount").show()

+-------+------------------+-----------------+
|summary|          Landsize|    Propertycount|
+-------+------------------+-----------------+
|  count|             17982|            27244|
|   mean| 593.4889333778223|7566.781089414183|
| stddev|3757.2664286434324|4492.382417842941|
|    min|               0.0|             83.0|
|    max|          433014.0|          21650.0|
+-------+------------------+-----------------+



In [115]:
df_result.count()

res89: Long = 27247


### Group By and  Aggregation

In [116]:
df_result.groupBy("Suburb").agg(max("Price")).show()

+----------------+----------+
|          Suburb|max(Price)|
+----------------+----------+
|  Brunswick West| 2600000.0|
| South Melbourne| 4600000.0|
|    Ivanhoe East| 3850000.0|
|    Princes Hill| 2980000.0|
|      Cranbourne|  910500.0|
|         Ashwood| 2100000.0|
|       Brunswick| 2545000.0|
|South Kingsville| 1320000.0|
|        Brighton|    1.12E7|
|        Oak Park| 1453000.0|
|         Doveton|  707000.0|
|       Albanvale|  655000.0|
|      Brookfield|  555000.0|
|        Lynbrook|  630000.0|
|     Ferny Creek|  690000.0|
|     Pascoe Vale| 1663000.0|
| Blackburn North| 1570000.0|
|     Sandringham| 4600000.0|
|   Botanic Ridge|  750000.0|
|          Carrum|  980000.0|
+----------------+----------+
only showing top 20 rows



In [117]:
df_result.groupBy("Suburb").agg(min("Price")).show()

+----------------+----------+
|          Suburb|min(Price)|
+----------------+----------+
|  Brunswick West|  237000.0|
| South Melbourne|  320000.0|
|    Ivanhoe East|  405000.0|
|    Princes Hill|  938000.0|
|      Cranbourne|  490000.0|
|         Ashwood|  535000.0|
|       Brunswick|  170000.0|
|South Kingsville|  300000.0|
|        Brighton|  290000.0|
|        Oak Park|  358500.0|
|         Doveton|  396000.0|
|       Albanvale|  415000.0|
|      Brookfield|  445000.0|
|        Lynbrook|  597500.0|
|     Ferny Creek|  690000.0|
|     Pascoe Vale|  248500.0|
| Blackburn North|  590000.0|
|     Sandringham|  430000.0|
|   Botanic Ridge|  750000.0|
|          Carrum|  435000.0|
+----------------+----------+
only showing top 20 rows



In [118]:
df_result.groupBy("Distance").agg(round(mean("Price"),0)).show()

+--------+--------------------+
|Distance|round(avg(Price), 0)|
+--------+--------------------+
|    14.9|            740966.0|
|    13.4|           1222423.0|
|    15.5|            976895.0|
|    15.4|           1000869.0|
|     2.4|           1064972.0|
|     8.0|           1116152.0|
|    10.2|           1600977.0|
|    24.7|            628671.0|
|    32.3|            713722.0|
|     0.0|            858388.0|
|    17.9|            832302.0|
|    43.3|            594750.0|
|    11.4|           1111867.0|
|     5.4|           1761224.0|
|    23.8|            600357.0|
|    16.6|            564100.0|
|     7.0|            974803.0|
|    11.5|            776150.0|
|     3.5|           1346543.0|
|    31.7|            504432.0|
+--------+--------------------+
only showing top 20 rows



### Correlation

 Finding out correlations between "Price" and attributes:

|-- Rooms
|-- Distance
|-- Rooms
|-- Bathroom
|-- Car
|-- Landsize
|-- BuildingArea
|-- YearBuilt
|-- Propertycount
--|

In [119]:
df_result.select(corr("Rooms","Price")).show()

+-------------------+
| corr(Rooms, Price)|
+-------------------+
|0.46523834510759615|
+-------------------+



In [120]:
import org.apache.spark.sql.functions.corr
df_result.select(corr("Distance","Price")).show()

+---------------------+
|corr(Distance, Price)|
+---------------------+
| -0.21138434279157942|
+---------------------+



import org.apache.spark.sql.functions.corr


In [121]:
df_result.select(corr("Bathroom","Price")).show()

+---------------------+
|corr(Bathroom, Price)|
+---------------------+
|   0.4298780777015672|
+---------------------+



In [122]:
df_result.select(corr("Car","Price")).show()

+-------------------+
|   corr(Car, Price)|
+-------------------+
|0.20180256061576263|
+-------------------+



In [123]:
df_result.select(corr("Landsize","Price")).show()

+---------------------+
|corr(Landsize, Price)|
+---------------------+
| 0.032748365249470925|
+---------------------+



In [124]:
df_result.select(corr("BuildingArea","Price")).show()

+-------------------------+
|corr(BuildingArea, Price)|
+-------------------------+
|       0.1007536394731018|
+-------------------------+



In [125]:
df_result.select(corr("YearBuilt","Price")).show()

+----------------------+
|corr(YearBuilt, Price)|
+----------------------+
|   -0.3333055641267079|
+----------------------+



In [126]:
df_result.select(corr("Propertycount","Price")).show()

+--------------------------+
|corr(Propertycount, Price)|
+--------------------------+
|      -0.05901668608374418|
+--------------------------+



### Select relevant features:

In [127]:

df_result = df_result.select("Price","Method","Type","Distance","Rooms","Bathroom","Car","Landsize","Propertycount", "Suburb","Street","Date")

df_result: org.apache.spark.sql.DataFrame = [Price: double, Method: string ... 10 more fields]


#### Filtering null values

In [128]:
val df_not_null = df_result.na.drop
df_not_null.count()

df_not_null: org.apache.spark.sql.DataFrame = [Price: double, Method: string ... 10 more fields]
res101: Long = 17701


In [129]:
df_not_null.printSchema()


root
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Rooms: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- Propertycount: double (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Date: string (nullable = true)



#### Write down clean data:

In [130]:
! hadoop fs -mkdir -p /tmp/output

In [131]:
! hadoop fs -ls /tmp/

Found 3 items


drwxrwx---   - root supergroup          0 2020-05-22 07:18 /tmp/hadoop-yarn


drwxr-xr-x   - root supergroup          0 2020-05-22 07:24 /tmp/output


drwxr-xr-x   - root supergroup          0 2020-05-22 07:20 /tmp/rs_in




In [132]:
df_not_null
   .coalesce(1)
   .write
   .format("csv")
   .option("header","true")
   .mode("overwrite").option("sep",",")
   .save("hdfs://localhost:9000/tmp/output")

In [133]:
! hadoop fs -ls -R /tmp/output

-rw-r--r--   3 root supergroup          0 2020-05-22 07:24 /tmp/output/_SUCCESS


-rw-r--r--   3 root supergroup    1285556 2020-05-22 07:24 /tmp/output/part-00000-521117aa-7453-48dc-979b-96800950d356-c000.csv




Save the clean data to disk

In [134]:
! hadoop fs -copyToLocal /tmp/output/\*.csv ./../data-clean/mh.csv

In [135]:
! ls ./../data-clean/

mh.csv



## References

Apache Spark (n.d.). _Spark Scala API (Scaladoc). Overview._ https://spark.apache.org/docs/latest/api/java/overview-summary.html

Apache Spark (n.d.). _Basic Statistic._ https://spark.apache.org/docs/latest/ml-statistics.html

Bahadoor N. (2020). _Spark Tutorials_ https://allaboutscala.com/big-data/spark/#dataframe-statistics-correlation

Databricks. (2020). _Introduction to DataFrames - Scala._  https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-scala.html 

Grimaldi E. (2018). _Pandas vs. Spark: how to handle dataframes (Part II.)_  https://towardsdatascience.com/python-pandas-vs-scala-how-to-handle-dataframes-part-ii-d3e5efe8287d 

Phatak M. (2016). _Statistical Data Exploration using Spark 2.0 - Part 2 : Shape of Data with Histograms_ http://blog.madhukaraphatak.com/statistical-data-exploration-spark-part-2/